In [ ]:
def isSoundtrack(styles):
    if "Soundtrack" in styles:
        return True
    return False

def getStyles(vals):
    retvals = []
    styles = vals.get("Style")
    if styles is None:
        return retvals
    elif isinstance(styles, list):
        retvals = [x[0] for x in styles]

    return retvals

def getArtists(vals):
    retvals = [x[0] for x in vals]
    return retvals

### Get Soundtracks
tmp = df[df['Profile'].apply(getStyles).apply(isSoundtrack) == True]

### Set Artists
df["Artists"] = df['Artist'].apply(getArtists)

In [ ]:
soundtracks = sorted(list(set(tmp["Album"].to_list())))

term = "Walt Disney's "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Soundtrack Of "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]

terms = []
terms.append(" (Original Cast Sound Track)")
terms.append(" (Original Cast Soundtrack)")
terms.append(" (Music From The Original Motion Picture Sound Track)")
terms.append(" (Music From The Soundtrack)")
terms.append(" (Music from the soundtrack of)")
terms.append(" (Original Motion Picture Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (The Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Soundtrack Recording)")
terms.append(" (Music From The Motion Picture Sound Track)")
terms.append(" (Music From The Motion Picture Soundtrack)")
terms.append(":  Music From The Original Motion Picture Soundtrack")
terms.append(" - Music From The Original Motion Picture Soundtrack")
terms.append(" (Music From The Motion Picture)")
terms.append(" (An Original Soundtrack Recording)")
terms.append("  (An Original Soundtrack Recording)")
terms.append(" Original Motion Picture Soundtrack")

terms+=['(Original Television Soundtrack Recording)',
 '(Motion Picture Soundtrack)',
 '(The Original Motion Picture Soundtrack)',
 "(The Original Motion Picture Soundtrack)",
 '(Music From The Original Motion Picture Soundtrack)',
 '(Original Motion Picture Soundtrack Album)',
 "(Music From The Original Motion Picture Soundtrack)",
 "(Original TV Soundtrack)",
 "(From The Motion Picture Soundtrack)",
 '(Soundtrack)',
 '(Limited Event Series Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music Composed For The Original Motion Picture Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(The Original Soundtrack Recording)',
 '(original Soundtrack From The Movie)',
 '(The Original Movie Soundtrack)',
 '(The Original Soundtrack From The Motion Picture)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music From The Original Soundtrack)',
 '(From The Original Motion Picture Soundtrack)'
 ' - Original Soundtrack Recording',
 ' (An Original Sound Track Recording)',
 ' - Music from the soundtrack of the Motion Picture',
 ' Original Soundtrack',
 ' Original Cast Sound Track Album',
 ' - Original Cast Sound Track Album',
 ' (From The Original Sound Track)',
 ' - Music From The Original Motion Picture Sound Track',
 ' (The Original Motion Picture Sound Track)',
 ' - Original Sound Track Recording',
 ': Original Cast Soundtrack',
 ':  Original Cast Soundtrack',
 ' (Original Cast Sound Track Album)']
for term in terms:
    soundtracks = [x.replace(term, "").strip() if x.endswith(term) else x for x in soundtracks]

print(len(soundtracks))
soundtracks = set(soundtracks)
print(len(soundtracks))
soundtracks

In [ ]:
[y for y in [x for x in soundtracks if x.find("Soundtrack") != -1] if y.find("(") != -1]

In [ ]:
def getStyles(vals):
    retvals = []
    if vals is None:
        return retval
    if isinstance(vals, str):
        vals = json.loads(vals)
    if isinstance(vals, dict):
        styles = vals.get("Style")
        if styles is None:
            return retvals
        elif isinstance(styles, list):
            retvals = [x[0] for x in styles]
            
    return retvals

In [ ]:
x = str(vals)
json.loads(x)
#getStyles(vals)

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-08-01 23:50:40.689217


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col    = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


In [14]:
for modVal in range(93,95):
    arts.parseArtistModValFiles(modVal=modVal, force=True)

Parsing Artist Files For ModVal 93
Forcing Reloads of ModVal=93
  Processing 3819 files.
	Processed 0/3819 files.
	Processed 250/3819 files.
	Processed 500/3819 files.
	Processed 750/3819 files.
	Processed 1000/3819 files.
	Processed 1250/3819 files.
	Processed 1500/3819 files.
	Processed 1750/3819 files.
	Processed 2000/3819 files.
	Processed 2250/3819 files.
	Processed 2500/3819 files.
	Processed 2750/3819 files.
	Processed 3000/3819 files.
	Processed 3250/3819 files.
	Processed 3500/3819 files.
	Processed 3750/3819 files.
Saving 3819 new artist IDs to /Volumes/Music/Discog/artists-db/93-DB.p
  --> This file is 6.7MB.
Parsing Artist Files For ModVal 94
Forcing Reloads of ModVal=94
  Processing 3842 files.
	Processed 0/3842 files.
	Processed 250/3842 files.
	Processed 500/3842 files.
	Processed 750/3842 files.
	Processed 1000/3842 files.
	Processed 1250/3842 files.
	Processed 1500/3842 files.
	Processed 1750/3842 files.
	Processed 2000/3842 files.
	Processed 2250/3842 files.
	Processe

In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [11]:

refs = [dbCol.get('489428')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

REF --> /artist/489428-AyalÃ©w-MÃ¨sfin
/Volumes/Music/Discog/artists/28/489428.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/489428-Ayal%C3%83%C2%A9w-M%C3%83%C2%A8sfin?sort=year%2Casc&limit=500
  File ID != Artist ID. Renaming to /Volumes/Music/Discog/artists/11/1973611.p
Saving /Volumes/Music/Discog/artists/11/1973611.p
  --> This file is 21.8kB.
Done. Sleeping for 2 seconds


In [ ]:
vals = ["1166600", "1324400", "1340600", "15800", "1626700", "1691500", "2006300", "211200", "23500", "2360300", "2701000", "29300", "40000", "4300900", "437700", "444000", "4731500", "563200", "6400", "72800", "90800"]

In [ ]:
modVal=0

artistDBDir = disc.getArtistsDBDir()
dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
dbdata  = getFile(dbname)
print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))

for val in vals:
    del dbdata[val]

savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
print("Saving {0} new artist IDs to {1}".format(len(dbdata), savename))
saveJoblib(data=dbdata, filename=savename, compress=True)


In [ ]:
for modVal in range(0,100):
    try:
        arts.assertDBModValExtraData(modVal)
    except:
        pass

In [5]:
vals="""Downloading: https://www.discogs.com/artist/302000-Ballard-MacDonald?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/75910-Stephen-W-Tayler?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/11415-Bob-Clearmountain?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/834222-Paul-Myers-2?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/674125-Ram%C3%83%C2%B3n-Arcusa?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/253127-Fredrik-Nordstr%C3%83%C2%B6m?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/449529-Denny-Purcell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/273544-Angus-Young?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/1641558-Jacques-Fournier?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/252962-Alfred-Lion?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/244167-Brendan-OBrien?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/27468-Bill-Bottrell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/382969-Fred-Kevorkian?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/142073-Bernard-Edwards-Nile-Rodgers?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/255681-Bill-Szymczyk?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/332586-Ralph-Bass?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/180298-Tom-Dowd?sort=year%2Casc&limit=500&page=2"""

In [27]:

artistDBDir = disc.getArtistsDBDir()

/artist/67340-Yaki-Da

urls = vals.split('\n')
for url in urls:
    ref = url.split('?')[0]


    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(url)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)
    
    url = ref.replace("Downloading: ", "")
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
    sleep(1)

/Volumes/Music/Discog/artists/0/302000.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/302000-Ballard-MacDonald
Saving /Volumes/Music/Discog/artists/0/302000.p
  --> This file is 17.1kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/10/75910.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/75910-Stephen-W-Tayler
Saving /Volumes/Music/Discog/artists/10/75910.p
  --> This file is 17.6kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/15/11415.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/11415-Bob-Clearmountain
Saving /Volumes/Music/Discog/artists/15/11415.p
  --> This file is 17.8kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/22/834222.p exists.
Now Downloading in Artists(): https://www.discogs.com/artist/834222-Paul-Myers-2
Saving /Volumes/Music/Discog/artists/22/834222.p
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
/Volumes/Music/Discog/artists/25/674125.p exists.


In [ ]:
for modVal in range(1,100):
    try:
        arts.assertDBModValData(modVal)
    except:
        pass

In [ ]:
for modVal in [1]:
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
data = getFile("/Volumes/Music/Discog/artists-db/1-DB.p")

In [ ]:
artistIDtoRefData = disc.getArtistIDToRefData()

In [ ]:
for modVal in range(20,60):
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
x = str(artistData.url.url)

In [ ]:
x.Contains("x")

In [ ]:
errs=getFile("errs.p")
print("Found {0} err files".format(len(errs)))
from collections import Counter
n = 0
for item in Counter(refCounts).most_common():
    n += 1
    ref      = item[0]
    cnts     = item[1]
    if errs.get(ref):
        continue
    discID   = arts.discogsUtils.getArtistID(ref)
    url      = arts.getArtistRef(ref)
    savename = arts.getArtistSavename(discID)
    if isFile(savename):
        continue
    try:
        arts.downloadArtistURL(url, savename)
        sleep(1)
    except:
        print("Adding {0} to errs".format(ref))
        errs[ref] = True
        if len(errs) % 10 == 0:
            saveFile(idata=errs, ifile="errs.p", debug=True)
        pass
    
    print("----------> {0} \t {1}".format(cnts,n)
    print("")



In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

def f(a_list):
    out = None
    for n in a_list:
        dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
        dbdata = getFile(dbname, version=3)
        return dbdata
        if out is None:
            out = dbdata
        else:
            out.update(dbdata)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return result.get()
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
f_amp(range(100))

In [ ]:
from searchUtils import findExt

artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

totalSaves = 0
for i in range(maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')
    break

    #dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
    #dbdata = getFile(dbname, version=3)

In [ ]:
len(files)

In [ ]:
chunks = [files[:10][i::5] for i in range(5)]

In [ ]:
[len(x) for x in chunks]

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col = collections(disc)

In [ ]:
from time import sleep
stracks = getFile("soundtracks.p")
from numpy import ceil
for cnt,style in stracks.items():
    print(cnt,'\t',style)
    maxmp = int(ceil(int(cnt.replace(',', ""))/250))
    col.downloadCollection(maxPages=min([15,maxmp]), style=["Soundtrack",style], master=True) #, fmat=fmat)

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()   

In [ ]:
def f(a_list, dbdata):
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
    
    
    dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
    dbdata = getFile(dbname, version=3)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return sum(result.get())
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
a1 = "Duke Ellington"
a2 = "Coleman Hawkins"

In [ ]:
disc     = discogs()
artistDB = disc.getArtistVariationNameToIDsData()

In [ ]:
artistDB[a1]

In [ ]:
artistDB[a2]

In [ ]:
artistIDToName = disc.getArtistIDToNameData()
artistIDToRef = disc.getArtistIDToRefData()

In [ ]:
artistIDToRef['284747']

In [ ]:
artistIDToName['284747-1']

In [ ]:
from artists import artists
from artist import artist

In [ ]:
art = artist()
arts = artists(disc)

In [ ]:
ifile = '/Volumes/Music/Discog/artists/47/284747.p'
info = art.getData(ifile)

In [ ]:
len(info['Media']["Albums"])

In [ ]:
discID   = '284747'
href     = artistIDToRef[discID]
url      = arts.getArtistRef(href)
savename = arts.getArtistSavename(discID)
print(url)
url="{0}/?sort=year%2Casc&limit=500".format(url)
arts.downloadArtistURL(url, savename, force=True)

In [ ]:
html = str(getFile(ifile))
type(html)
html.find("1,221")

In [ ]:
len(html)

In [ ]:
info.keys()

for key in info.keys():
    print("\n\n\n{0}".format(key))
    val = info[key]
    if isinstance(val, dict):
        for key2,val2 in val.items():
            if isinstance(val2, list):
                print("\t{0}".format(key2))
                for key3 in val2:
                    print("\t\t{0}".format(key3))
            elif isinstance(val2, dict):
                print("\t{0}".format(key2))
                for key3,val3 in val2.items():
                    print("\t\t{0}".format(key3))
                    print("\t\t\t{0}".format(val3))
            else:
                print("\t{0}:\t{1}".format(key2,val2))
    else:
        print(info[key])

In [ ]:
a = artist()

In [ ]:
x = a.getData(ifile)

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
x['']

In [ ]:
apc.err

In [ ]:
apc.__dict__

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
pages

In [ ]:
tot

In [ ]:
int(ceil(tot/ppp))

In [ ]:
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()

artistIDToRef = disc.getArtistIDToRefData()

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()
toget={}
extras={}
totalSaves = 0
for i in range(19,maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if toget.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref = info.url.url
            
        pages  = info.pages
        if pages.redo is True:
            toget[artistID] = ref
            if len(toget) % 25 == 0:
                print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref)
                
    saveFile(idata=toget, ifile="toget3.p")

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()

redo={}
more={}
extras={}
totalSaves = 0
for i in range(1, 8):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if redo.get(artistID) is not None:
            continue
        if more.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref    = info.url.url
        pages  = info.pages
        if pages.redo is True:
            redo[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())
            
        if pages.more is True:
            more[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())

                
    #saveFile(idata=toget, ifile="toget.p")

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

for artistID,artistRef in more.items():
    ifile = arts.getArtistSavename(artistID)
    info  = art.getData(ifile)
    pages = info.pages
    
    npages = pages.pages

    print(ifile,'\t',artistRef,'\t',pages.get())

    for p in range(2, npages+1):
        url      = arts.getArtistURL(artistRef, p)
        savename = arts.getArtistSavename(artistID, p)
        if not isFile(savename):
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

In [ ]:
tmp = {k: redo[k] for k in ['1028302', '1323902', '1411102', '200402', '2159702', '228202']}
tmp

In [ ]:
redo = tmp.copy()
redo

In [ ]:
redo

In [ ]:
%load_ext autoreload
%autoreload

redo={"780701": "/artist/780701-Whitfield-Strong"}

from artist import artist
from artists import artists

from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()
arts = artists(disc)

from time import sleep
for artistID,artistRef in redo.items():
    url = arts.getArtistURL(artistRef)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, force=True)
    sleep(1)